In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import tensorflow as tf
import cv2


In [3]:
from tensorflow import keras
#print(keras.__version__)

In [4]:
data_directory = 'data'
data = tf.keras.utils.image_dataset_from_directory(data_directory)

Found 34 files belonging to 3 classes.


In [5]:
type(data)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [6]:
data_iterator = data.as_numpy_iterator()

In [7]:
type(data_iterator)

tensorflow.python.data.ops.dataset_ops._NumpyIterator

In [8]:
batch = data_iterator.next()

In [9]:
type(batch)

tuple

In [10]:
batch

(array([[[[135.       , 125.96855  ,  46.       ],
          [135.       , 125.81131  ,  46.       ],
          [135.       , 126.       ,  46.       ],
          ...,
          [ 44.       ,  22.894531 ,  14.894531 ],
          [ 45.       ,  24.       ,  15.       ],
          [ 45.       ,  24.       ,  15.       ]],
 
         [[136.       , 126.03516  ,  46.035156 ],
          [136.       , 126.31641  ,  46.316406 ],
          [136.       , 126.05562  ,  46.05562  ],
          ...,
          [ 43.       ,  21.367188 ,  13.       ],
          [ 43.       ,  21.683594 ,  13.316406 ],
          [ 43.       ,  21.683594 ,  13.316406 ]],
 
         [[137.       , 127.       ,  47.       ],
          [137.       , 127.       ,  47.       ],
          [137.       , 127.       ,  47.       ],
          ...,
          [ 43.       ,  20.092697 ,  13.092697 ],
          [ 45.89453  ,  23.894531 ,  16.894531 ],
          [ 44.45604  ,  22.45604  ,  15.456039 ]],
 
         ...,
 
         [[ 

In [11]:
# Scale the data
data = data.map(lambda x,y: (x/255,y))

In [12]:
data.as_numpy_iterator().next()

(array([[[[0.01568628, 0.03137255, 0.02745098],
          [0.03039216, 0.04607843, 0.04215686],
          [0.01960784, 0.03529412, 0.03137255],
          ...,
          [0.04509804, 0.08431373, 0.0882353 ],
          [0.05098039, 0.09019608, 0.09411765],
          [0.05882353, 0.09019608, 0.09803922]],
 
         [[0.00882353, 0.0245098 , 0.02058824],
          [0.02352941, 0.03921569, 0.03529412],
          [0.02254902, 0.0382353 , 0.03431373],
          ...,
          [0.04117647, 0.08039216, 0.08431373],
          [0.05392157, 0.09313726, 0.09705883],
          [0.05784314, 0.08921569, 0.09705883]],
 
         [[0.02941176, 0.04509804, 0.04117647],
          [0.02058824, 0.03627451, 0.03235294],
          [0.00882353, 0.0245098 , 0.02058824],
          ...,
          [0.04803922, 0.0872549 , 0.09117647],
          [0.05882353, 0.09411765, 0.1       ],
          [0.04803922, 0.07941177, 0.0872549 ]],
 
         ...,
 
         [[0.35588235, 0.39509803, 0.43039215],
          [0.28431

In [13]:
#5. Split Data
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)


In [14]:
# 6. Build DL model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [15]:
model = Sequential()

In [16]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [17]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

In [19]:
# Train
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [20]:
hist = model.fit(train, epochs=10, validation_data=val, callbacks =[tensorboard_callback])

Epoch 1/10


1/1 [==============================] - 2s 2s/step - loss: 0.7719 - accuracy: 0.3125
Epoch 2/10
1/1 [==============================] - 1s 544ms/step - loss: -0.5891 - accuracy: 0.3750
Epoch 3/10
1/1 [==============================] - 1s 548ms/step - loss: 0.7455 - accuracy: 0.3438
Epoch 4/10
1/1 [==============================] - 1s 524ms/step - loss: -1.3552 - accuracy: 0.3438
Epoch 5/10
1/1 [==============================] - 1s 513ms/step - loss: 0.5007 - accuracy: 0.3438
Epoch 6/10
1/1 [==============================] - 1s 574ms/step - loss: -1.8902 - accuracy: 0.3125
Epoch 7/10
1/1 [==============================] - 0s 499ms/step - loss: -0.6623 - accuracy: 0.3438
Epoch 8/10
1/1 [==============================] - 1s 519ms/step - loss: -7.3379 - accuracy: 0.3750
Epoch 9/10
1/1 [==============================] - 1s 503ms/step - loss: -4.2607 - accuracy: 0.3438
Epoch 10/10
1/1 [==============================] - 1s 518ms/step - loss: -2.0918 - accuracy: 0.3750


In [21]:
img = cv2.imread('IMG_1973.jpg')
plt.imshow(img)
plt.show()

: 